In [1]:
import math
import pandas as pd
import numpy as np
import sqlalchemy
import sys
print(pd.__version__)

pd.set_option('max_columns', None)

#from postgressql import *

1.3.4


In [2]:
conn_params_dic = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

sqlpath = "./sqlCommands/loadMastrDBfiles.sql"

def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'

def build_postgres_conn_string (param:dict) -> str:
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict):
    conString = build_postgres_conn_string(param)
    engine = sqlalchemy.create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict):# -> pandas.DataFrame:
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pd.read_sql(script, engine)

    return df

# Read from MaStR Postgres DB

In [3]:
sqlpath = "./sqlCommands/loadMastrDBfiles.sql"
df = read_postgres_from_queryfile(sqlpath, conn_params_dic)
#df = df.dropna()
df['SpezifischeLeistung'] = (df.Nettonennleistung * 1000) / ((math.pi * df.Rotordurchmesser**2) / 4)
df.head(n=10)#.filter(regex="grad(?i)").head()
#df.to_excel()

,EinheitMastrNummer,DatumLetzteAktualisierung,LokationMaStRNummer,NetzbetreiberpruefungStatus,NetzbetreiberpruefungDatum,AnlagenbetreiberMastrNummer,Land,Bundesland,Landkreis,Gemeinde,Gemeindeschluessel,Postleitzahl,Gemarkung,FlurFlurstuecknummern,StrasseNichtGefunden,Hausnummer_nv,HausnummerNichtGefunden,Ort,Laengengrad,Breitengrad,Registrierungsdatum,Inbetriebnahmedatum,EinheitSystemstatus,EinheitBetriebsstatus,NichtVorhandenInMigriertenEinheiten,DatumDesBetreiberwechsels,DatumRegistrierungDesBetreiberwechsels,NameStromerzeugungseinheit,Weic_nv,Kraftwerksnummer_nv,Energietraeger,Bruttoleistung,Nettonennleistung,AnschlussAnHoechstOderHochSpannung,FernsteuerbarkeitNb,FernsteuerbarkeitDv,FernsteuerbarkeitDr,Einspeisungsart,GenMastrNummer,Lage,Hersteller,Technologie,Typenbezeichnung,Nabenhoehe,Rotordurchmesser,Rotorblattenteisungssystem,AuflageAbschaltungLeistungsbegrenzung,EegMaStRNummer,AuflagenAbschaltungSchallimmissionsschutzNachts,AuflagenAbschaltungSchallimmissionsschutzTagsueber,AuflagenAbschaltungSchattenwurf,AuflagenAbschaltungTierschutz,AuflagenAbschaltungEiswurf,AuflagenAbschaltungSonstige,Weic,WeicDisplayName,Seelage,ClusterNordsee,Wassertiefe,Kuestenentfernung,GeplantesInbetriebnahmedatum,Strasse,Hausnummer,Adresszusatz,DatumEndgueltigeStilllegung,ClusterOstsee,DatumBeginnVoruebergehendeStilllegung,Kraftwerksnummer,DatumWiederaufnahmeBetrieb,SpezifischeLeistung
0,SEE900002935310,2021-10-19 10:28:04.490695,SEL957516775208,1,2021-10-27,ABR935787334270,84,1400.0,Oder-Spree,Jacobsdorf,12067237.0,15236.0,Pillgram,03:106,0.0,0.0,0.0,Jacobsdorf,14.387865,52.351393,2021-08-20,2021-07-15,472,35,0,2020-03-30,2020-05-04,WEA 3,0,0,2497,4500.00,4500.00,1.0,0.0,1.0,0.0,688.0,SGE970229452754,888,1627.0,691,N149/4.0-4.5NH164,164.00,149.1,1.0,1.0,EEG949357649497,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT,257.731408
1,SEE900004001082,2021-01-27 20:20:59.429862,SEL949397946808,1,2021-01-27,ABR967616635029,84,1409.0,Paderborn,Büren,5774016.0,33154.0,Salzkotten,Flur 2 und Flurstück 192,0.0,0.0,0.0,Salzkotten,8.651723,51.699230,2019-07-11,2011-03-10,472,35,0,NaT,NaT,BroWi SalzkottenII,1,0,2497,2000.00,2000.00,0.0,1.0,1.0,0.0,688.0,SGE953172981724,888,1586.0,691,E-82,108.38,82.0,NaN,0.0,EEG905689504088,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT,378.714915
2,SEE900011187731,2020-01-08 18:07:22.967045,SEL901928237038,1,2020-01-10,ABR962772458710,84,1410.0,Eifelkreis Bitburg-Prüm,Hütten,7232056.0,54673.0,Hütten,1 89/1,0.0,0.0,0.0,Hütten,6.248464,50.018301,2020-01-08,2002-12-20,472,35,0,NaT,NaT,Hütten-Wind Heide,1,0,2497,600.00,600.00,0.0,1.0,0.0,1.0,688.0,None,888,1586.0,691,E40 6.44,78.00,44.0,0.0,0.0,EEG940019359143,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT,394.599032
3,SEE900013535608,2021-01-21 15:14:32.282210,SEL956642007177,1,2020-10-29,ABR955714105537,84,1410.0,Donnersbergkreis,Gauersheim,7333022.0,67294.0,Gauersheim,"900/1, 901",0.0,0.0,0.0,Gauersheim,8.045690,49.651650,2019-02-14,2013-12-27,472,35,0,NaT,NaT,V202217,0,0,2497,3075.00,3075.00,1.0,1.0,1.0,1.0,688.0,SGE938072733246,888,1660.0,691,Vestas V112,140.00,112.0,NaN,1.0,EEG998437166812,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT,312.118272
4,SEE900014803461,2021-11-15 14:18:58.767802,SEL977165118843,0,2021-11-15,ABR918453648000,84,1415.0,Erfurt,Erfurt,16051000.0,99095.0,None,None,NaN,NaN,NaN,Erfurt,NaN,NaN,2021-03-25,2021-03-25,472,37,0,NaT,NaT,Kleinwindrad,0,0,2497,2.02,2.02,NaN,0.0,NaN,NaN,689.0,None,888,2554.0,692,Tesup Atlas2.0,NaN,1.2,0.0,0.0,EEG918287727568,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaT,None,None,None,NaT,NaN,NaT,None,NaT,1786.072139
5,SEE900018842459,2019-06-06 07:29:04.554597,SEL957663731027,1,2019-06-06,ABR974422971418,84,1413.0,Nordsachsen,Wiedemar,14730340.0,4509.0,Zaasch,Flur 4 Flurstück 6/1,0.0,0.0,0.0,Wiedemar,12.286509,51.558769,2019-06-06,1999-12-22,472,3

In [4]:
print(f"Size: {format_bytes(sys.getsizeof(df))}")
print(f"Size: {format_bytes(df.memory_usage(index=True, deep=True).sum())}")

df.isna().sum()#.count()

Size: (43.666871070861816, 'megabytes')
Size: (43.666855812072754, 'megabytes')


EinheitMastrNummer                           0
DatumLetzteAktualisierung                    0
LokationMaStRNummer                       2090
NetzbetreiberpruefungStatus                  0
NetzbetreiberpruefungDatum                1730
                                         ...  
ClusterOstsee                            32602
DatumBeginnVoruebergehendeStilllegung    32795
Kraftwerksnummer                         32833
DatumWiederaufnahmeBetrieb               32830
SpezifischeLeistung                        398
Length: 70, dtype: int64

In [5]:
len(df.columns)

70

# PLOTTING SPACIAL DATA

In [6]:

import matplotlib.pyplot as plt
import plotly.express as px
import math

In [7]:
px.colors.named_colorscales()

['aggrnyl',
 'agsunset',
 'blackbody',
 'bluered',
 'blues',
 'blugrn',
 'bluyl',
 'brwnyl',
 'bugn',
 'bupu',
 'burg',
 'burgyl',
 'cividis',
 'darkmint',
 'electric',
 'emrld',
 'gnbu',
 'greens',
 'greys',
 'hot',
 'inferno',
 'jet',
 'magenta',
 'magma',
 'mint',
 'orrd',
 'oranges',
 'oryel',
 'peach',
 'pinkyl',
 'plasma',
 'plotly3',
 'pubu',
 'pubugn',
 'purd',
 'purp',
 'purples',
 'purpor',
 'rainbow',
 'rdbu',
 'rdpu',
 'redor',
 'reds',
 'sunset',
 'sunsetdark',
 'teal',
 'tealgrn',
 'turbo',
 'viridis',
 'ylgn',
 'ylgnbu',
 'ylorbr',
 'ylorrd',
 'algae',
 'amp',
 'deep',
 'dense',
 'gray',
 'haline',
 'ice',
 'matter',
 'solar',
 'speed',
 'tempo',
 'thermal',
 'turbid',
 'armyrose',
 'brbg',
 'earth',
 'fall',
 'geyser',
 'prgn',
 'piyg',
 'picnic',
 'portland',
 'puor',
 'rdgy',
 'rdylbu',
 'rdylgn',
 'spectral',
 'tealrose',
 'temps',
 'tropic',
 'balance',
 'curl',
 'delta',
 'oxy',
 'edge',
 'hsv',
 'icefire',
 'phase',
 'twilight',
 'mrybm',
 'mygbm']

In [19]:
start = '2015-10-15'
end = '2022-08-15'

test = df[(df["Inbetriebnahmedatum"] >= start) & (df["Inbetriebnahmedatum"] < end) & (df["Nettonennleistung"] < 6000)]

fig = px.scatter_mapbox(test, 
            lat='Breitengrad', 
            lon='Laengengrad', 
            #animation_frame = 'Inbetriebnahmedatum',
            color="Nettonennleistung", #"SpezifischeLeistung",
            #size="Nettonennleistung",
            hover_name="EegMaStRNummer", 
            hover_data=["Inbetriebnahmedatum","Hersteller",'Typenbezeichnung', "Nettonennleistung"],
            #color_discrete_sequence=px.colors.sequential.Rainbow, #px.colors.qualitative.Antique,
            color_continuous_scale="jet", 
            zoom=5,
            height=600,
            mapbox_style="open-street-map")

fig.update_layout(
    margin=dict(l=0, r=20, t=0, b=0)
)


fig.show()